<a href="https://colab.research.google.com/github/callmesukhi/BFVM19PROG1/blob/main/assessments/Assignment_week_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 04 Assignment weather data

Welcome to week four of this course programming 1. You will to organise your data into the required format and apply smoothing. In this assignment we will work with weatherdata from the KNMI. A subset of weatherdata is for you available in the file: `KNMI_20181231.csv`. The data consist of several stations with daily weather data of several years. Your task is to make a plot similar to the plot below.


<img src="https://github.com/callmesukhi/BFVM19PROG1/blob/main/assessments/images/weather.png?raw=1" alt="drawing" width="400"/>


Furthermore the plot needs the following enhancements

1. proper titles and ticks
2. a slider widget selecting a particular year or all years
3. lines need to be smoothed
3. legends needs to be added

Use your creativity. Consider colors, alpha settings, sizes etc. 

Learning outcomes

- load, inspect and clean a dataset 
- reformat dataframes
- apply smoothing technologies
- visualize timeseries data

The assignment consists of 6 parts:

- [part 1: load the data](#0)
- [part 2: clean the data](#1)
- [part 3: reformat data](#2)
- [part 4: smooth the data](#3)
- [part 5: visualize the data](#4)
- [part 6: Challenge](#5)

Part 1 and 5 are mandatory, part 6 is optional (bonus)
To pass the assingnment you need to a score of 60%. 


---

<a name='0'></a>
## Part 1: Load the data

Either load the dataset `KNMI_20181231.csv` or `KNMI_20181231.txt.tsv`. The dataheaders contain spaces and are not very self explainable. Change this into more readable ones. Select data from station 270. Select only the mean, minimum and maximum temperature. The data should look something like this:


In [2]:
import pandas as pd
import numpy as np
import yaml

In [12]:
df = pd.read_csv('/content/KNMI_20181231.txt.tsv', sep=',', skiprows=63, parse_dates=['YYYYMMDD'])
df = df.drop(df.index[0])
df.rename(columns={"# STN":"STATION", "YYYYMMDD":"DATE", "   TG":"Tmean", "   TN":"Tmin", "   TX":"Tmax"}, inplace=True)
df = df.drop(['   SQ', '   DR', '   RH'], axis='columns')
df_270 = df[df.STATION == 270]
df_270.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,2,3,4,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATION,DATE,Tmean,Tmin,Tmax
97642,270,2000-01-01,42,-4,79
97643,270,2000-01-02,55,33,74
97644,270,2000-01-03,74,49,89
97645,270,2000-01-04,46,22,75
97646,270,2000-01-05,41,14,56


        station      Date  Tmean   Tmin   Tmax
245205      270  19510101      0    -40     24
245206      270  19510102      9     -5     19
245207      270  19510103      5     -8     17
245208      270  19510104      5    -15     17
245209      270  19510105     26      6     46

---

# temp range for few years
# min of each month temp for one particular year
# max of each 
# average temp of each day 
# user slider for smoothing 
# user slider for selecting different years 

<a name='1'></a>
## Part 2: Clean the data

The data ia not clean. There are empty cells in the dataframe which needs to be replaced with NaN's and the temperature is in centidegrees which needs to be transformed into degrees. The date field needs a datetime format. For visualization convience we would like to remove the leap year. Conduct the cleaning.

In [24]:
df_270['DATE'] = pd.to_datetime(df_270['DATE'], errors='coerce')
#replace cells with spaces to NaN
# df_270.replace(r'\s+', np.nan, regex=True)
#change data formats
df[['STATION','Tmean','Tmin','Tmax']] = df[['STATION','Tmean','Tmin','Tmax']].apply(pd.to_numeric, errors='coerce')
# df_270['STATION'] = pd.to_numeric(df_270.STATION, errors='coerce')
#change temperatures to celcius degrees
#remove leap 
df_270.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6940 entries, 97642 to 104581
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   STATION  6940 non-null   int64         
 1   DATE     6940 non-null   datetime64[ns]
 2   Tmean    6940 non-null   object        
 3   Tmin     6940 non-null   object        
 4   Tmax     6940 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 325.3+ KB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>pd.to_datetime(df['Date'].astype(str), format='%Y%m%d')</li>
    <li>regex for empty cells = `^\s*$` </li>
    <li>remove month == 2 & day == 29</li> 
</ul>
</details>

In [ ]:
#Test your outcome
df.info()

### Expected outcome

DatetimeIndex: 24820 entries, 1951-01-01 to 2018-12-31
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   station  24820 non-null  int64         
 1   Date     24820 non-null  datetime64[ns]
 2   Tmean    24819 non-null  float64       
 3   Tmin     24819 non-null  float64       
 4   Tmax     24819 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 1.1 MB

---

<a name='2'></a>
## Part 3: Reform your data

First we will split the data in data from 2018 and data before 2018. Best is to split this in two dataframes. 
Next we need for the non 2018 data the minimum values for each day and the maximum values for each day. So we look for the minimum value out of all january-01 minimum values (regardless the year). Create a dataframe with 365 days containing the ultimate minimum and the ultimate maximum per day. 


In [ ]:
def month_day(df_multipleyears):
    #your code to reform data here
    
    print(df_groupedbymonthday)

In [ ]:
#Test your code
def test_reformed(df)
    #
    df = df[(df.index.year > 2007) & (df.index.year < 2018)]
    month_day(df)

test_reformed(df)

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use the dt.month and dt.day to groupby</li>
</ul>
</details>

### Expected outcome
Note, the layout or names my differ, but the length should be 365 and the minimum values should be the same

month  day
1      1      -5.8
       2      -7.5
       3     -12.6
       4      -4.1
       5      -6.0
              ... 
12     27     -4.8
       28     -4.2
       29     -6.7
       30    -10.2
       31    -10.6
Name: Tmin, Length: 365, dtype: float64

---

<a name='3'></a>
## Part 4: Smooth the data

Make a function that takes an array or a dataframe column and returns an array of smoothed data. Explain in words why you choose a certain smoothing algoritm


In [ ]:
#your code here
#your motivation here

---

<a name='4'></a>
## Part 5: Visualize the data

Plot the mean temperature of the year 2018. Create a shaded band with the ultimate minimum values and the ultimate maximum values from the multi-year dataset. Add labels, titles and legends. Use proper ranges. Be creative to make the plot attractive. 



<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hints</b></font>
</summary>
<ul><li>use from bokeh.models import Band</li>
    <li>use ColumnDataSource to parse data arrays</li>
    <li>look for xaxis tick formatters</li>
</ul>
</details>

---

<a name='5'></a>
## Part 6: Challenge

Make a widget in which you can select the year range for the multiyear set. Add this to your layout to make the plot interactive. Add another widget to select or deselect the smoother. Inspiration: https://demo.bokeh.org/weather